**Assignment4, PartC:** Demonstrate an example where fenicsx fails.

A 2D rectangular beam is fixed at one end and a distributed force is applied to that beam. .

Problem: I used the 2D linear elasticity problem. The parameters are taken from [here](https://comet-fenics.readthedocs.io/en/latest/demo/elasticity/2D_elasticity.py.html). The following tutorial is taken from [this link](https://bleyerj.github.io/comet-fenicsx/intro/linear_elasticity/linear_elasticity.html). ChatGPT was used in commenting, getting the correct notations and writing fucntions.

**Importing modules**

In [1]:
import numpy as np                              # Numerical operations
import matplotlib.pyplot as plt                  # Plotting library
import logging                                   # For logging progress messages
from ufl import sym, grad, Identity, tr, inner, Measure, TestFunction, TrialFunction  # UFL for FEM formulation
from mpi4py import MPI                           # MPI for parallel computing
from dolfinx import fem                          # DOLFINx FEM tools
import dolfinx.fem.petsc                         # PETSc linear solver interface
from dolfinx.mesh import create_rectangle, CellType  # Mesh generation tools

**Defining Functions**

In [2]:
# --- Setup Logging ---
logging.basicConfig(level=logging.INFO, format='%(message)s')  # Configure logging format and level

# --- Core Functions ---

# Function to generate a rectangular mesh domain
def generate_rectangle_domain(length, height, Nx, Ny, cell_type):
    return create_rectangle(MPI.COMM_WORLD, [np.array([0, 0]), np.array([length, height])], [Nx, Ny], cell_type=cell_type)

# Function to define function space and displacement function
def set_function_space(domain, degree):
    dim = domain.topology.dim                         # Get dimension of the domain (2D here)
    shape = (dim,)                                    # Vector field shape (displacement vector)
    V = fem.functionspace(domain, ("P", degree, shape))  # Define vector function space of degree 'degree'
    u_sol = fem.Function(V, name="Displacement")      # Initialize displacement function
    return V, u_sol, dim

# Function to define material properties (Lamé parameters)
def define_material_properties(domain, E_value, nu_value):
    E = fem.Constant(domain, E_value)                 # Young's modulus as constant
    nu = fem.Constant(domain, nu_value)               # Poisson's ratio as constant
    lmbda = E * nu / (1 + nu) / (1 - 2 * nu)          # First Lamé parameter
    mu = E / 2 / (1 + nu)                             # Second Lamé parameter (shear modulus)
    return E, nu, lmbda, mu

# Define strain tensor (ε)
def epsilon(v): 
    return sym(grad(v))                               # Symmetric gradient of displacement

# Define stress tensor (σ) using Hooke's law for isotropic material
def sigma(v, lmbda, mu, dim): 
    return lmbda * tr(epsilon(v)) * Identity(dim) + 2 * mu * epsilon(v)

# Define weak form (variational problem)
def define_variational_problem(V, domain, lmbda, mu, rho, g, dim):
    u = TrialFunction(V)                              # Trial function (unknown displacement)
    v = TestFunction(V)                               # Test function
    f = fem.Constant(domain, np.array([0, -rho * g])) # Body force vector (gravity)
    dx = Measure("dx", domain=domain)                 # Integration measure
    a = inner(sigma(u, lmbda, mu, dim), epsilon(v)) * dx  # Bilinear form
    L = inner(f, v) * dx                              # Linear form
    return a, L

# Apply Dirichlet BC: Fix left edge (u = 0)
def apply_boundary_conditions(V):
    left = lambda x: np.isclose(x[0], 0.0)            # Define left boundary condition (x = 0)
    left_dofs = fem.locate_dofs_geometrical(V, left)  # Locate degrees of freedom on left edge
    return [fem.dirichletbc(np.zeros((2,)), left_dofs, V)]  # Apply zero displacement BC

# Solve linear system
def solve_problem(a, L, u_sol, bcs):
    problem = fem.petsc.LinearProblem(a, L, u=u_sol, bcs=bcs, petsc_options={"ksp_type": "preonly", "pc_type": "lu"})  # Direct solver
    return problem.solve()                             # Solve and return solution

# Extract vertical displacement at the tip (right edge, mid-height)
def extract_tip_displacement(V, uh, length, height):
    V1, dof_map = V.sub(1).collapse()                  # Extract y-component function space
    sub_tip = fem.locate_dofs_geometrical(V1, lambda x: np.isclose(x[0], length) & np.isclose(x[1], height/2))  # Locate tip DOF
    global_tip = dof_map[sub_tip[0]]                   # Map to global DOF index
    return uh.x.array[global_tip]                      # Return displacement value at tip

# Theoritical deflection calculation
def compute_theoretical_deflection(length, height, E, nu, rho, g):
    f_vol = - rho * g                          # Body force per unit volume due to gravity
    I = height**4 / 12                         # Second moment of area for a rectangular cross-section
    delta_th = float(f_vol * length**4 / (8 * E * I))  # Theoretical tip deflection (without correction)
    E_effective = E / (1 - nu ** 2)            # Effective Young's modulus for plane strain condition
    delta_th_corrected = float(f_vol * length**4 / (8 * E_effective * I))  # Corrected theoretical deflection
    return delta_th, delta_th_corrected        # Return both uncorrected and corrected deflections


**Main Execution**

In [3]:
# --- Define Simulation Parameters ---
length, height = 25.0, 1.0        # Beam dimensions
Nx, Ny = 250, 10                  # Number of elements along length and height
degree = 2                        # Polynomial degree of finite element basis functions
cell_type = CellType.quadrilateral # Type of mesh cell (quadrilateral elements)
E_value = 1e5                     # Young's modulus
nu_value = 0.5           # Poisson's ratio
rho = 2e-3                        # Material density
g = 9.81                          # Gravitational acceleration

# --- Generate Mesh Domain ---
domain = generate_rectangle_domain(length, height, Nx, Ny, cell_type)

# --- Define Function Space for Displacement Solution ---
V, u_sol, dim = set_function_space(domain, degree)

# --- Define Material Properties (including Lame parameters) ---
E, nu, lmbda, mu = define_material_properties(domain, E_value, nu_value)

# --- Define Variational Problem (weak form of PDE) ---
a, L = define_variational_problem(V, domain, lmbda, mu, rho, g, dim)

# --- Apply Boundary Conditions (e.g., fixing one side of the beam) ---
bcs = apply_boundary_conditions(V)

# --- Solve the Finite Element Problem ---
uh = solve_problem(a, L, u_sol, bcs)

# --- Extract Displacement at the Beam Tip ---
tip_disp = extract_tip_displacement(V, uh, length, height)

# --- Compute Theoretical Deflections (basic and corrected for plane strain) ---
delta_th, delta_th_corrected = compute_theoretical_deflection(length, height, float(E), float(nu), rho, g)

# --- Print Results and Errors ---
print(f"Numerical tip displacement: {tip_disp:.6e}")  # FEM solution
print(f"Theoretical tip displacement: {delta_th:.6e}")  # Basic theory
print(f"Theoretical tip displacement(corrected): {delta_th_corrected:.6e}")  # Corrected theory
print(f"Relative error:           {abs(tip_disp - delta_th)/abs(delta_th):.2%}")  # Error w.r.t basic theory
print(f"Relative error (corrected theory):           {abs(tip_disp - delta_th_corrected)/abs(delta_th_corrected):.2%}")  # Error w.r.t corrected theory


/projectnb/me700/students/sjazad/.conda/envs/hw4-env/lib/python3.13/site-packages/dolfinx/mesh.py:867: RuntimeWarning: mpi4py.MPI.Session size changed, may indicate binary incompatibility. Expected 32 from C header, got 40 from PyObject
  msh = _cpp.mesh.create_rectangle_float64(comm, points, n, cell_type, partitioner, diagonal)

/projectnb/me700/students/sjazad/.conda/envs/hw4-env/lib/python3.13/site-packages/dolfinx/fem/function.py:630: RuntimeWarning: mpi4py.MPI.Session size changed, may indicate binary incompatibility. Expected 32 from C header, got 40 from PyObject
  cpp_dofmap = _cpp.fem.create_dofmap(mesh.comm, mesh.topology._cpp_object, cpp_element)

/projectnb/me700/students/sjazad/.conda/envs/hw4-env/lib/python3.13/site-packages/dolfinx/la.py:367: RuntimeWarning: mpi4py.MPI.Session size changed, may indicate binary incompatibility. Expected 32 from C header, got 40 from PyObject
  return PETSc.Vec().createGhost(ghosts, size=size, bsize=bs, comm=map.comm)  # type: ignore



Numerical tip displacement: nan
Theoretical tip displacement: -1.149609e-01
Theoretical tip displacement(corrected): -8.622070e-02
Relative error:           nan%
Relative error (corrected theory):           nan%


**Why ν = 0.5 Messes Up the Stiffness Matrix**

In linear elasticity the global stiffness matrix $K$ is assembled from the bilinear form  
$$
a(u,v) = \int_\Omega \bigl[\lambda\,\mathrm{tr}(\varepsilon(u))\,\mathrm{tr}(\varepsilon(v))
+ 2\mu\,\varepsilon(u):\varepsilon(v)\bigr]\,\mathrm{d}\Omega,
$$  
where the Lamé parameters are  
$$
\mu = \frac{E}{2(1 + \nu)}, 
\quad
\lambda = \frac{E\,\nu}{(1 + \nu)\,(1 - 2\nu)}.
$$  
When $\nu = 0.5$, the denominator $(1 - 2\nu)$ becomes zero, causing $\lambda$ to approach infinity. As a result:

- The volumetric stiffness term $\lambda\,\mathrm{tr}(\varepsilon)^2$ dominates, effectively “locking” the elements (basically makes it “infinitely hard” for the element to change its volume).  
- When λ → ∞ , some parts of the global stiffness matrix K become extremely large or infinite. Numerically, that means K has nearly zero pivots (it’s almost singular) or even inconsistent signs, so you can’t invert or solve with it reliably.

